In [0]:
#### Devolve um data frame incluindo a o timestamp na última coluna
from pyspark.sql.functions import current_timestamp

def add_ingestion_date(input_df):
    output_df = input_df.withColumn("ingestion_date", current_timestamp())
    return output_df

In [0]:
#### Elimina uma partição de uma tabela caso a tabela/partição já exista
# Usamos a função collect que retorna os dados em memória para apagar a partição se a tabela e partição já existir
def drop_partition_if_exists(input_df, db_name, table_name, column_partition):
    for item_list in input_df.select(f"{column_partition}").distinct().collect():
        
        if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
        
            spark.sql(f"ALTER TABLE {db_name}.{table_name} DROP IF EXISTS PARTITION ({column_partition} = '{item_list[column_partition]}')")

In [0]:
### Função para validar se o arquivo existe
def valida_arquivo(folder_path, file_date, file_name):
    file_path = f"{folder_path}/{file_date}/{file_name}"
    try:
        dbutils.fs.ls(file_path)
        print(f"Processando arquivo: {file_path}")
    except Exception as e:
        dbutils.notebook.exit(f"Arquivo não encontrado no caminho: {file_path}")

In [0]:
### Função para validar se a pasta e arquivos existem
from fnmatch import fnmatch

def valida_pasta_arquivo(folder_path, file_date, subfolder, file_name_pattern):
    # Monta o caminho completo
    folder_full_path = f"{folder_path.rstrip('/')}/{file_date}/{subfolder.strip('/')}/"

    try:
        # Tenta listar os arquivos da pasta
        files = dbutils.fs.ls(folder_full_path)
    except Exception as e:
        # Trata erro de pasta inexistente
        if "FileNotFoundException" in str(e):
            dbutils.notebook.exit(f"Pasta não encontrada: {folder_full_path}")
        else:
            dbutils.notebook.exit(f"Erro ao acessar a pasta: {folder_full_path} - {str(e)}")
    
    # Filtra arquivos que casam com o padrão
    matching_files = [f.path for f in files if fnmatch(f.name, file_name_pattern)]

    if matching_files:
        for path in matching_files:
            print(f"Processando arquivo: {path}")
    else:
        dbutils.notebook.exit(
            f"Nenhum arquivo encontrado com padrão '{file_name_pattern}' em {folder_full_path}"
        )
#Exemplo de uso
#valida_pasta_arquivo("/mnt/moviehistoryadilmor/bronze", "2024-12-24", "production_company", "production_company_*.csv")

In [0]:
### Função para efetuar merge de uma data frame com uma tabela delta final
def merge_delta_lake(input_df, db_name, table_name, folder_path, merge_condition, partition_column):
    
    from delta.tables import DeltaTable

    if (spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):

        deltaTable = DeltaTable.forPath(spark, f"{folder_path}/{table_name}")

        deltaTable.alias("tgt") \
        .merge(
            input_df.alias("src"),
            merge_condition
            ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()

    else:
        input_df.write.mode("overwrite").partitionBy(partition_column).format("delta").saveAsTable(f"{db_name}.{table_name}")